In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from bidi.algorithm import get_display
import arabic_reshaper
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.isri import ISRIStemmer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder  


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('court_project', index_col=0)

In [4]:
data.head(3)

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,...,appeal_id,appeal_date,appeal_text,links1,links1-href,القضايا,القضايا-href,نص الحكم,الصفحات,الصفحات-href
0,1640298117-87,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38991,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1640297824-62,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38903,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1640298107-86,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38992,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.court.value_counts()

المحكمة: المحكمة التجارية                    113
المحكمة: المحكمة العليا -  الهيئة الدائمة     44
المحكمة: المحكمة العامة                       35
Name: court, dtype: int64

In [6]:
data['court'] =data['court'].str.replace('المحكمة: المحكمة التجارية', 'التجارية')
data['court'] =data['court'].str.replace('المحكمة: المحكمة العليا -  الهيئة الدائمة', 'العليا')
data['court'] =data['court'].str.replace('المحكمة: المحكمة العامة', 'العامة')

In [7]:
data.court.value_counts()

التجارية    113
العليا       44
العامة       35
Name: court, dtype: int64

In [8]:
data[data['judgment_text'].isnull()]

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,...,appeal_id,appeal_date,appeal_text,links1,links1-href,القضايا,القضايا-href,نص الحكم,الصفحات,الصفحات-href
152,1640286767-5,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٨٧٨٠ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39114,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
153,1640286772-7,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٣١٤٥ لعام ١٤٤١ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39022,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
154,1640286803-18,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ١٦٥٦٣ لعام ١٤٤٠ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38911,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,9,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
155,1640286790-13,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٣١٤ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38927,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,9,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
156,1640286777-9,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٥٨٥٧ لعام ١٤٤٠ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39013,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
157,1640286770-6,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٥٣٢٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39105,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
158,1640286782-11,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٥٥٨٤ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38999,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
159,1640286812-21,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٨٧٩٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38870,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,9,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
160,1640286757-2,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٧٦٥١ لعام ١٤٤١ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/32005,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,»»,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
161,1640286780-10,https://sjp.moj.gov.sa/Filter/Index?subIDS=600...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,القضية رقم ٢١٨٦ لعام ١٤٤١ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39012,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,10,https://sjp.moj.gov.sa/Filter/FilterDataPager?...


In [9]:
data.dropna(subset=["judgment_text"],inplace = True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 213
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      192 non-null    object
 1   web-scraper-start-url  192 non-null    object
 2   links                  102 non-null    object
 3   links-href             102 non-null    object
 4   court                  192 non-null    object
 5   case_number            102 non-null    object
 6   case_date              102 non-null    object
 7   appellate_court        89 non-null     object
 8   appellate_id           89 non-null     object
 9   appellate_date         89 non-null     object
 10  region                 116 non-null    object
 11  city                   192 non-null    object
 12  judgment_text          192 non-null    object
 13  appeal                 89 non-null     object
 14  pages                  92 non-null     object
 15  pages-href             

In [11]:
data.drop(['نص الحكم','links1','links0-href','links0','appeal_text','appeal_date','appeal_id','appeal_court','court_date',
          'court_id','links1-href'], axis = 1,inplace=True)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 213
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      192 non-null    object
 1   web-scraper-start-url  192 non-null    object
 2   links                  102 non-null    object
 3   links-href             102 non-null    object
 4   court                  192 non-null    object
 5   case_number            102 non-null    object
 6   case_date              102 non-null    object
 7   appellate_court        89 non-null     object
 8   appellate_id           89 non-null     object
 9   appellate_date         89 non-null     object
 10  region                 116 non-null    object
 11  city                   192 non-null    object
 12  judgment_text          192 non-null    object
 13  appeal                 89 non-null     object
 14  pages                  92 non-null     object
 15  pages-href             

In [13]:
data['الصفحات-href'].isnull().sum()

192

In [14]:
data['القضايا-href'].isnull().sum()

192

In [15]:
data['pages-href'].isnull().sum()

100

In [16]:
data['pages'].isnull().sum()

100

In [17]:
data.drop(['pages','pages-href','القضايا-href','الصفحات-href'], axis = 1,inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 213
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      192 non-null    object
 1   web-scraper-start-url  192 non-null    object
 2   links                  102 non-null    object
 3   links-href             102 non-null    object
 4   court                  192 non-null    object
 5   case_number            102 non-null    object
 6   case_date              102 non-null    object
 7   appellate_court        89 non-null     object
 8   appellate_id           89 non-null     object
 9   appellate_date         89 non-null     object
 10  region                 116 non-null    object
 11  city                   192 non-null    object
 12  judgment_text          192 non-null    object
 13  appeal                 89 non-null     object
 14  القضايا                0 non-null      object
 15  الصفحات                

In [19]:
data['القضايا'].isnull().sum()

192

In [20]:
data['الصفحات'].isnull().sum()

192

In [21]:
data.drop(['القضايا','الصفحات'], axis = 1,inplace=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 213
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      192 non-null    object
 1   web-scraper-start-url  192 non-null    object
 2   links                  102 non-null    object
 3   links-href             102 non-null    object
 4   court                  192 non-null    object
 5   case_number            102 non-null    object
 6   case_date              102 non-null    object
 7   appellate_court        89 non-null     object
 8   appellate_id           89 non-null     object
 9   appellate_date         89 non-null     object
 10  region                 116 non-null    object
 11  city                   192 non-null    object
 12  judgment_text          192 non-null    object
 13  appeal                 89 non-null     object
dtypes: object(14)
memory usage: 22.5+ KB


In [23]:
data.head(4)

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,region,city,judgment_text,appeal
0,1640298117-87,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38991,التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
1,1640297824-62,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38903,التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
2,1640298107-86,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38992,التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...
3,1640297908-69,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٣٣١ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39268,التجارية,رقم القضية - القرار: ٤٣٣١,تاريخها: ١٨/١١/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٣٧,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...


## remove ponctuations

In [24]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|"!”…“–ـ'''

In [25]:
def remove_punctuations(text):
    translator = str.maketrans(' ', ' ', arabic_punctuations)
    return text.translate(translator)
data['judgment_text'] =data['judgment_text'].map(remove_punctuations)

In [26]:
#data.judgment_text = data.judgment_text.replace('\n','')

## normalize_arabic

In [27]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [28]:
data['judgment_text'] =data['judgment_text'].map(normalize_arabic)

## remove numbers

In [29]:
data['judgment_text'] =data['judgment_text'].str.replace('\w*\d\w*', ' ')

In [30]:
data['judgment_text'] =data['judgment_text'].str.replace('\\', ' ')

## remove \n

In [31]:
data['judgment_text'] =data['judgment_text'].str.replace(r'\s', ' ')

In [32]:
data['judgment_text'] =data['judgment_text'].str.replace(r'\s*[A-Za-z]+\b', ' ')

In [33]:
data['judgment_text'] =data['judgment_text'].str.replace('-', ' ')

In [34]:
data['judgment_text'] =data['judgment_text'].str.replace('\u200c', ' ')

In [35]:
data['judgment_text'][1]

'الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد فلدي الدائره التجاريه الثانيه عشره وبناءً علي القضيه رقم    لعام   ه  المقامه من شركه شبال للتجاره مساهمه مقفله سجل تجاري   ضد عبدﷲ محمد عبدﷲ فلاته هويه وطنيه    القاضي بندر بن صالح الحميد رئيسا القاضي بكر بن خلف العنزي عضوا القاضي عبدالرحمن بن سعد السعد عضوا   الوقائع  تتحصل وقائع هذه الدعوي بالقدر اللازم لاصدار هذا الحكم وحسبما يتبين من اوراقها بتقدم وكيل المدعي نايف بدر غازي المرشدي هويه رقم  بصفته وكيلا عن شركه شبال للتجاره مساهمه مقفله بوكاله رقم    بلائحه دعوي ضد المدعي عليها مؤسسه اثير الصفوه للتجاره سجل رقم  تضمنت ما يلي    بتاريخ       م قامت المدعي عليها بفتح حساب شراء بضاعه بالاجل لدي موكلتي علي ان توفر موكلتي للمدعي عليها الاقمشه المطلوبه بالاجل وتقوم المدعي عليها بالسداد خلال العام نفسه ولا يتم تجاوز نفس العام    استمرت العلاقه التعاقديه طيله الفتره الماضيه بناءً علي الاتفاق نفسه مع تغيير المؤسسه المباشره للتعاملات الي مؤسسه اثير الصفوه للتجاره والعائده جميعها للمالك نفسه عبدﷲ محمد عبدﷲ فلاته    اشترت المدعي عليها اقمشه رجاله ص

Strip tatweel from a text and return a result text

In [36]:
text="حَكَمَتِ"
a = text.replace('\u064E', '')# َ
print(a)
a=a.replace('\u0650', '')#ِ
print(a)

حكمتِ
حكمت


In [37]:
import pyarabic.araby as araby
import pyarabic.number as number

In [38]:
araby.strip_tashkeel('حَكَمَتِ')

'حكمت'

In [39]:
#araby.strip_tashkeel(data['judgment_text'])

In [40]:
'''
def converter(x):
    try:
        return ([araby.strip_tashkeel(x) for i in x])
    except AttributeError:
        return None  # or some other value
data['judgment_text'] = data['judgment_text'].str.apply(converter)
'''

"\ndef converter(x):\n    try:\n        return ([araby.strip_tashkeel(x) for i in x])\n    except AttributeError:\n        return None  # or some other value\ndata['judgment_text'] = data['judgment_text'].str.apply(converter)\n"

In [41]:
'''
def remove_tashkeel(f):
    return araby.strip_tashkeel(f)
data['judgment_text'] = data.judgment_text.map(remove_tashkeel)
'''

"\ndef remove_tashkeel(f):\n    return araby.strip_tashkeel(f)\ndata['judgment_text'] = data.judgment_text.map(remove_tashkeel)\n"

In [42]:
data.judgment_text = data.judgment_text.str.replace('\u0640', '')

In [43]:
data.judgment_text= data.judgment_text.str.replace('\u064E', '')# َ

In [44]:
data.judgment_text= data.judgment_text.str.replace('\u0650', '')# َ

In [45]:
 def get_Verdict(Verdict):
        
    if "حكمت"  in Verdict:
         return str.split(Verdict, "حكمت",1)[1]
    #if "حـكمـت"  in Verdict:
        # return str.split(Verdict, "حـكمـت",1)[1]
    if "قررت"  in Verdict:
         return str.split(Verdict, "قررت",1)[1]
    if "منطوق"  in Verdict:
         return str.split(Verdict, "منطوق",1)[1]
    if "منطوف"  in Verdict:
         return str.split(Verdict, "منطوف",1)[1]
    return None
data['Verdict'] =data['judgment_text'].map(get_Verdict)

In [46]:
data['Verdict'][200]

' الدائره بعدم اختصاص المحكمه التجاريه نوعيًا بنظر الدعوي رقم   لعام   والمقامه من المدعي قاسم بن حسين بن علي المكحل ذي الهويه الوطنيه رقم ضد المدعي عليها فاطمه بنت حسين الشيخ ذي الهويه الوطنيه رقم  وﷲ الموفق والهادي الي سواء السبيل وصلي ﷲ وسلم علي نبينا محمد وعلي اله وصحبه اجمعين  عضو فيصل بن احمد الزهراني  عضو عبدالعزيز بن عبدالرحمن بن صالح الراجح  رئيس الدائره عبدﷲ بن عبدالرحمن اليابس'

In [47]:
#data['judgment_text'][29]

In [48]:
blob = TextBlob("منطوف")
print(blob.correct())

منطوف


In [49]:
data[data['Verdict'].isnull()]

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,region,city,judgment_text,appeal,Verdict


In [50]:
'''
for col in data['Verdict']:
    data['Verdict'].to_csv('Verdict.csv')
    '''

"\nfor col in data['Verdict']:\n    data['Verdict'].to_csv('Verdict.csv')\n    "

In [51]:
'''
for col in data['judgment_text']:
    data['judgment_text'].to_csv('judgment_csv')
    '''

"\nfor col in data['judgment_text']:\n    data['judgment_text'].to_csv('judgment_csv')\n    "

In [52]:
'''
with open("judgment_csv") as f1:
    set1 = set(f1.readlines())
with open("Verdict.csv") as f2:
    set2 = set(f2.readlines())

nondups = set1 - set2

with open("file1.csv", "w") as out:
    out.writelines(nondups)
    '''

'\nwith open("judgment_csv") as f1:\n    set1 = set(f1.readlines())\nwith open("Verdict.csv") as f2:\n    set2 = set(f2.readlines())\n\nnondups = set1 - set2\n\nwith open("file1.csv", "w") as out:\n    out.writelines(nondups)\n    '

In [53]:
#txt = pd.read_csv('file1.csv')

In [54]:
data['judgment_text'][88]

'الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد فلدي الدائره التجاريه الاولي وبناءً علي القضيه رقم    لعام   ه  المقامه من صاحبه السمو الملكي الاميره ريم بنت فيصل بن سعود بن عبدالعزيز ال سعود هويه وطنيه   ضد سعيد احمد سعيد المساعدي الشهري هويه وطنيه      القاضي ابراهيم بن محمد الفالح رئيسا القاضي عبدﷲ بن محمد البدر عضوا القاضي ناصر بن عبدﷲ بن مبرد عضوا   الوقائع  تتحصل وقائع هذه القضيه بالقدر اللازم لاصدار هذا الحكم بان وكيل المدعيه تقدم بلائحه دعوي الي المحكمه التجاريه بالرياض جاء فيها ما نصه  ان المدعي عليه تقدم للمحكمه التجاريه بدعوي ضد موكلتي صاحبه مؤسسه اميره الخليج الدعائيه لاقامه وتنظيم المعارض سجل تجاري رقم   وتتخلص وقائع الدعوي بمطالبته بمبلغ مائه وخمسون الف ريال واستند في دعواه علي العقد المؤرخ في   لمسابقه شاعر الوطن وسند القبض رقم   في تاريخ   واحيلت القضيه رقم   لعام   للدائره الثالثه عشر في تاريخ   وصدر فيها حكمًا في تاريخ   بمنطوقه فسخ الشراكه و الزام موكلتي بان تدفع للمدعي مبلغ مائه وخمسون الف ريال وتم تنفيذ الحكم من طرف موكلتي وطبقًا للماده الثانيه والعشرون بعد المائه من

In [55]:
data['Verdict'][88]

' الدائره بالزام المدعي عليه سعيد بن احمد بن سعيد المساعدي الشهري هويه وطنيه رقم  بان يسلم للمدعيه ريم بنت فيصل بن سعود ال سعود هويه وطنيه رقم  السندين الاتيين   المؤرخ في   المعنون بعقد اتفاق   سند القبض رقم   وتاريخ   لما هو مبين بالاسباب وبﷲ التوفيق  عضو  عبدﷲ بن محمد البدر عضو  ناصر بن عبدﷲ بن مبرد رئيس الدائره  ابراهيم بن محمد الفالح'

In [56]:
 def get_incident(incident):
        
    if "حكمت"  in incident:
         return str.split(incident, "حكمت",1)[0]
    #if "حـكمـت"  in incident:
        # return str.split(incident, "حـكمـت",1)[0]
    if "قررت"  in incident:
         return str.split(incident, "قررت",1)[0]
    if "منطوق"  in incident:
         return str.split(incident, "منطوق",1)[0]
    if "منطوف"  in incident:
         return str.split(incident, "منطوف",1)[0]
    return None
data['incident'] =data['judgment_text'].map(get_incident)

In [57]:
data['incident'][88]

'الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد فلدي الدائره التجاريه الاولي وبناءً علي القضيه رقم    لعام   ه  المقامه من صاحبه السمو الملكي الاميره ريم بنت فيصل بن سعود بن عبدالعزيز ال سعود هويه وطنيه   ضد سعيد احمد سعيد المساعدي الشهري هويه وطنيه      القاضي ابراهيم بن محمد الفالح رئيسا القاضي عبدﷲ بن محمد البدر عضوا القاضي ناصر بن عبدﷲ بن مبرد عضوا   الوقائع  تتحصل وقائع هذه القضيه بالقدر اللازم لاصدار هذا الحكم بان وكيل المدعيه تقدم بلائحه دعوي الي المحكمه التجاريه بالرياض جاء فيها ما نصه  ان المدعي عليه تقدم للمحكمه التجاريه بدعوي ضد موكلتي صاحبه مؤسسه اميره الخليج الدعائيه لاقامه وتنظيم المعارض سجل تجاري رقم   وتتخلص وقائع الدعوي بمطالبته بمبلغ مائه وخمسون الف ريال واستند في دعواه علي العقد المؤرخ في   لمسابقه شاعر الوطن وسند القبض رقم   في تاريخ   واحيلت القضيه رقم   لعام   للدائره الثالثه عشر في تاريخ   وصدر فيها حكمًا في تاريخ   بمنطوقه فسخ الشراكه و الزام موكلتي بان تدفع للمدعي مبلغ مائه وخمسون الف ريال وتم تنفيذ الحكم من طرف موكلتي وطبقًا للماده الثانيه والعشرون بعد المائه من

In [58]:
data[data['incident'].isnull()]

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,region,city,judgment_text,appeal,Verdict,incident


## Stemming

In [59]:
st = ISRIStemmer()
def streem(f):
    return st.stem(f)
#data['Verdict'] = data['Verdict'].map(streem)
data['incident'] = data['incident'].map(streem)

In [60]:
data['Verdict'][10]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

# Stop words 

In [61]:
stopwords = nltk.corpus.stopwords.words('arabic')

In [62]:
'و' in stopwords

True

In [63]:
word_st = 'الحمد','لله','تعالي','التوفيق','وصلي','الله','وسلم','علي','نبينا','محمد','اله','وصحبه','اجمعين','عضو','رئيس','الدائره','المحكمه','المحاكم','الحمد','والصلاه','والسلام','القضيه','رقم','لعام','سجل','القاضي','الموافق','المدعي','وكيل','الوكاله','المحاماه','المحاكم','بن','محكمة'

In [64]:
for i in range(len(word_st)):
    stopwords.append(word_st[i])

In [65]:
word_st2 = 'رسول','ﷲ','و','العقد','الحكم','الدعوي','للمحاكم','منطوقه','منطوق'

In [66]:
for i in range(len(word_st2)):
    stopwords.append(word_st2[i])

In [67]:
#data['Verdict'] = data['Verdict'].apply(lambda t: " ".join(word for word in t.split() if word not in stopwords))

In [68]:
data['Verdict'][100]

' الدائره  قبول الاعتراض شكلاً وفي الموضوع نقض الحكم الصادر من دائره الاستئناف الرابعه بالمحكمه التجاريه بالرياض في القضيه رقم   وتاريخ         واعاده الاوراق للمحكمه مصدرته للنظر في الدعوي من غير من نظرها علي ضوء ما ورد بهذا الحكم وبالله تعالي التوفيق  وصلي الله وسلم علي نبينا محمد وعلي اله وصحبه اجمعين   عضو محمد بن صالح اليحيي عضو فرحان بن يحيي الفيفي  رئيس الدائره عبدالوهاب بن محمد المنصوري'

In [69]:
data['incident'] = data['incident'].apply(lambda t: " ".join(word for word in t.split() if word not in stopwords))

In [70]:
data['incident'][70]

'حمد اما فلدي التجاريه الاولي وبناء المقامه سعيد عبدﷲ العمري هويه وطنيه ضد بندر القاسم هويه وطنيه ابراهيم الفالح رئيسا عبدﷲ سعد سعود العريفي عضوا عبدﷲ البدر عضوا الوقائع تتلخص وقائع ان وكاله عبدﷲ عبدﷲ الحارثي صاحب السجل المدني بموجب وتاريخ الصادره الخدمات الالكترونيه تقدم الي بصحيفه دعوي جاء مضمونها تم الاتفاق عقد شركه مضاربه بتاريخ اسلم مبلغ مائه الف ومده سنوات ونشاط الاستثمار مؤسسه للحام الحديد والالمنيوم باسم زوجته بنسبه ارباح المؤسسه التاريخ بدا بالعمل المتفق وكل نهايه سنه اطلب بيان الارباح وتاريخ نشوء الحق بتاريخ يوهمني دائما بعدم وجود الربح وعندما تاكد بعدم صحه يدعيه فقد اخبرني مدير المؤسسه ان ارباحها خلال الثلاثه سنوات الماضيه ثلاثمائه الف وحتي تاريخه الشراكه قائمه ولم تتم التصفيه ولذلك كله نطلب اثبات الشراكه وطالب دعواه اثبات شراكه تاريخ بنسبه وتوزيع الارباح وتقدر ارباح حصتي بمبلغ وقدره خمسمائه الف توزيع الارباح ونسبه حصته تقدر بمبلغ خمسمائه الف وبقيد قضيه واحالتها للدائره حددت لنظرها جلسه اليوم حضر عبدﷲ عبدﷲ الحارثي حضر اصاله بندر القاسم هويه وبسؤال المدعيه وكاله دعواه قال ورد

In [71]:
data['Verdict'][10]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [72]:
result = data['Verdict'].str.split('اختصاص المحكمه',expand=True)[1]

In [73]:
result.value_counts()

 التجاريه نوعيًا بنظر الدعوي رقم   لعام   والمقامه من المدعي قاسم بن حسين بن علي المكحل ذي الهويه الوطنيه رقم ضد المدعي عليها فاطمه بنت حسين الشيخ ذي الهويه الوطنيه رقم  وﷲ الموفق والهادي الي سواء السبيل وصلي ﷲ وسلم علي نبينا محمد وعلي اله وصحبه اجمعين  عضو فيصل بن احمد الزهراني  عضو عبدالعزيز بن عبدالرحمن بن صالح الراجح  رئيس الدائره عبدﷲ بن عبدالرحمن اليابس    10
 التجاريه بالرياض مكانياً بنظر هذه الدعوي وصلي ﷲ علي نبينا محمد وعلي اله وصحبه وسلم   عضو  عبدالعزيز بن سليمان الغيث عضو  يوسف بن محمد بن قاسم جعفري رئيس الدائره  جمال بن فاضل العمري                                                                                                                                                                                        2
 العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث                                                                                                                                                          

In [74]:
data['Verdict']

0       الدائره برفض الدعوي المقامه من شركه بيت الجزي...
1       الدائره بعدم قبول هذه الدعوي وﷲ الموفق وصلي ﷲ...
2       الدائره غيابيا بالزام خالد جمعه رضا حبش سجل م...
3       الدائره  برفض هذه الدعوي   عضو عبدالعزيز بن س...
4       الدائره بعدم جواز نظر هذه الدعوي لما هو مبين ...
                             ...                        
209     الدائره بالزام شركه ديبا العربيه السعوديه للم...
210     الدائره  قبول الاعتراض شكلاً وفي الموضوع نقض ...
211     الدائره رفع الجلسه للمداوله  الاسباب لما كانت...
212    ه الانظمه ذات الصله كما ان تقرير الاستحقاق ينب...
213     الدائره  قبول الاعتراض شكلاً وفي الموضوع نقض ...
Name: Verdict, Length: 192, dtype: object

In [75]:
data.court[3]

'التجارية'

In [79]:

def get_right_court(index, i):
    
    #for i in data['Verdict']:
    if "بعدم اختصاص"  in i:
            #data['right_court'] = data['Verdict'].str.split("من اختصاص المحكمه",1)[0]  
        if "من اختصاص المحكمه" in i:
            return str.split(i, "من اختصاص المحكمه",0)[0]
        else:
            return 2#None
    else:#if "اختصاص" not in i:
        return  data.loc[data.court[index]]


data['right_court'] =data['Verdict'].apply(lambda p: get_right_court(p, p.index))

TypeError: argument of type 'builtin_function_or_method' is not iterable

In [80]:

def get_right_court(index, i):
    
    #for i in data['Verdict']:
    if "بعدم اختصاص"  in i:
            #data['right_court'] = data['Verdict'].str.split("من اختصاص المحكمه",1)[0]  
        if "من اختصاص المحكمه" in i:
            return str.split(i, "من اختصاص المحكمه",0)[0]
        else:
            return 2#None
    else:#if "اختصاص" not in i:
        return  data.loc[data.court[index]]


data['right_court'] =data['Verdict'].apply(lambda p,index: get_right_court(p, p.index))


TypeError: <lambda>() missing 1 required positional argument: 'index'

In [81]:
data['right_court'].value_counts()

KeyError: 'right_court'

In [82]:
len(result)

192

In [83]:
result.fillna(data['court'])

0      التجارية
1      التجارية
2      التجارية
3      التجارية
4      التجارية
         ...   
209    التجارية
210      العليا
211      العامة
212      العليا
213      العليا
Name: 1, Length: 192, dtype: object

In [84]:
data['court'].value_counts()

التجارية    113
العليا       44
العامة       35
Name: court, dtype: int64

In [85]:
print(data['court'].iloc[[152]])

174    العليا
Name: court, dtype: object


In [86]:
f = data['court']

In [87]:
for i in range(len(f)):
    result.fillna(data['court'][i])

KeyError: 152

In [88]:
result.value_counts()

 التجاريه نوعيًا بنظر الدعوي رقم   لعام   والمقامه من المدعي قاسم بن حسين بن علي المكحل ذي الهويه الوطنيه رقم ضد المدعي عليها فاطمه بنت حسين الشيخ ذي الهويه الوطنيه رقم  وﷲ الموفق والهادي الي سواء السبيل وصلي ﷲ وسلم علي نبينا محمد وعلي اله وصحبه اجمعين  عضو فيصل بن احمد الزهراني  عضو عبدالعزيز بن عبدالرحمن بن صالح الراجح  رئيس الدائره عبدﷲ بن عبدالرحمن اليابس    10
 التجاريه بالرياض مكانياً بنظر هذه الدعوي وصلي ﷲ علي نبينا محمد وعلي اله وصحبه وسلم   عضو  عبدالعزيز بن سليمان الغيث عضو  يوسف بن محمد بن قاسم جعفري رئيس الدائره  جمال بن فاضل العمري                                                                                                                                                                                        2
 العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث                                                                                                                                                          

In [89]:
result

0      None
1      None
2      None
3      None
4      None
       ... 
209    None
210    None
211    None
212    None
213    None
Name: 1, Length: 192, dtype: object

In [90]:
result = []
for i in data['Verdict']:
    if "بعدم اختصاص" not in i:
        for j in data['court']:
            if i == j:
                result = data['court'][j] 
    
    if "بعدم اختصاص"  in i:
        if "من اختصاص المحكمه" in i:
            result = data['Verdict'][i].split('اختصاص المحكمه',expand=True)[1]
        else:
            result= None

KeyError: ' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [93]:
z = data['Verdict'][10]

In [94]:
z.split('اختصاص المحكمه')[1]

' العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [95]:
z

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [96]:
a="من اختصاص المحكمه"

In [97]:
z.split(a,0)[0]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [98]:
data['right_court'].value_counts()

KeyError: 'right_court'

In [99]:
data['Verdict'][10]

' الدائره  بعدم اختصاص المحاكم التجاريه نوعياً بنظر هذه الدعوي وهي من اختصاص المحكمه العامه  عضو يوسف بن محمد بن قاسم جعفري عضو بندر بن حمد بن علي التويجري رئيس الدائره عبدالعزيز بن سليمان الغيث'

In [101]:
data[data['Verdict'].str.contains('بعدم اختصاص')==True] 

,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,region,city,judgment_text,appeal,Verdict,incident
7,1640297166-4,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٨٧٩٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39115,التجارية,رقم القضية - القرار: ٨٧٩٧,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٢,تاريخه: ٥/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحاكم التجاريه نوعيا بن...,حمد اما فلدي التجاريه التاسعه عشره وبناء المقا...
10,1640297542-37,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٧٤٠٠ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39226,التجارية,رقم القضية - القرار: ٧٤٠٠,تاريخها: ١/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٥٧,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحاكم التجاريه نوعياً ...,حمد اما فلدي التجاريه الثالثه وبناء المقامه شر...
11,1640297281-14,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٣٨٩٥ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38926,التجارية,رقم القضية - القرار: ٣٨٩٥,تاريخها: ٢٣/٩/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٢٤٥,تاريخه: ٥/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحكمه التجاريه بالرياض...,حمد اما فلدي التجاريه الواحد والعشرون وبناء ال...
12,1640297772-57,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٧٤٠٠ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38981,التجارية,رقم القضية - القرار: ٧٤٠٠,تاريخها: ١/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٥٧,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحاكم التجاريه نوعياً ...,حمد اما فلدي التجاريه الثالثه وبناء المقامه شر...
23,1640297657-47,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٧٧٠١ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39007,التجارية,رقم القضية - القرار: ٧٧٠١,تاريخها: ١١/٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٩٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحكمه التجاريه نوعيًا ب...,حمد اما فلدي التجاريه الاولي وبناء المقامه جمع...
31,1640298076-83,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ١٠٦١٠ لعام ١٤٤١ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39019,التجارية,رقم القضية - القرار: ١٠٦١٠,تاريخها: ٢٧/٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ١٩٦٩,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص محاكم المملكه العربيه الس...,حمد اما فلدي التجاريه الرابعه وبناء المقامه شر...
48,1640297981-75,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٨٣٢١ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39169,التجارية,رقم القضية - القرار: ٨٣٢١,تاريخها: ٨/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٦٨٨,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاه والسلام علي رسول ﷲ اما بعد ف...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الدائره بعدم اختصاص المحكمه التجاريه بالرياض ...,حمد اما فلدي التجاريه الثالثه وبناء المقامه صا...
63,1640297354-21,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٨٧٩٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38870,التجارية,رقم القضية - القرار: ٨٧٩٧,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٢,تاريخه: ٥/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض